In [1]:
import sys

In [2]:
%matplotlib inline

In [9]:
from hgflask import app
import hgflask
from flask import jsonify, request

In [13]:
import hgtiles.bigwig as hgbi
import hgtiles.cooler as hgco

### HiGlass Server

In [7]:
@app.route('/api/v1/tileset_info/', methods=['GET'])
def tileset_info():
    uuids = request.args.getlist("d")

    info = {}
    for uuid in uuids:
        ts = next((ts for ts in hgflask.TILESETS if ts['uuid'] == uuid), None)
        
        if ts is not None:
            info[uuid] = ts.copy()
            if info[uuid]['filetype'] == 'bigwig':
                info[uuid].update(bigwig_tsinfo(ts['url']))                
        else:
            info[uuid] = {
                'error': 'No such tileset with uid: {}'.format(uuid)
            }
    return jsonify(info)


@app.route('/api/v1/tiles/', methods=['GET'])
def tiles():
    from flask import request, jsonify
    tids_requested = set(request.args.getlist("d"))
    if not tids_requested:
        return jsonify({'error': 'No tiles requested'})
    
    extract_uuid = lambda tid: tid.split('.')[0]
    uuids_to_tids = toolz.groupby(extract_uuid, tids_requested)
    
    tiles = []
    for uuid, tids in uuids_to_tids.items():
        ts = next( (ts for ts in hgflask.TILESETS if ts['uuid'] == uuid), None )
        if ts is not None:
            tiles.extend(bigwig_tiles(ts['url'], tids))

    data = {tid: tval for tid, tval in tiles}
    return jsonify(data)

In [8]:
hgflask.TILESETS = [{  
    'uuid': "abc",
    'filetype': "bigwig",
    'datatype': "vector",
    'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig',
    'private': False,
    'name': "blabla",
    'coordSystem': "hg19",
    'coordSystem2': "hg19",
    'created': "2017-02-02T21:10:56.265085Z",
}]

In [9]:
p = mp.Process(target=partial(app.run, debug=True, port=4000, host='0.0.0.0', use_reloader=False))
p.start()

 * Running on http://0.0.0.0:4000/ (Press CTRL+C to quit)
18.42.1.119 - - [30/Jan/2018 20:41:28] "GET /api/v1/tilesets/?limit=10000&dt=axis&dt=matrix&dt=vector&dt=multivec&dt=vector&dt=vector&dt=vector&dt=bed-value&dt=stacked-interval&dt=1d-projection&dt=gene-annotation&dt=2d-rectangle-domains&dt=chromsizes&dt=vector,stacked-interval,gene-annotation&dt=bedlike HTTP/1.1" 200 -
18.42.1.119 - - [30/Jan/2018 20:41:28] "GET /api/v1/tilesets/?limit=10000&dt=axis&dt=matrix&dt=vector&dt=multivec&dt=vector&dt=vector&dt=vector&dt=bed-value&dt=stacked-interval&dt=1d-projection&dt=gene-annotation&dt=2d-rectangle-domains&dt=chromsizes&dt=vector,stacked-interval,gene-annotation&dt=bedlike HTTP/1.1" 200 -
18.42.1.119 - - [30/Jan/2018 20:41:32] "GET /api/v1/tileset_info/?d=abc&s=BZSVkUdfRvyRE3TJWsveJw HTTP/1.1" 200 -
18.42.1.119 - - [30/Jan/2018 20:41:37] "GET /api/v1/tiles/?d=abc.13.877&d=abc.13.878&d=abc.13.879&s=BZSVkUdfRvyRE3TJWsveJw HTTP/1.1" 200 -
18.42.1.119 - - [30/Jan/2018 20:41:45] "GET /api